# Pictionary

In [1]:
import ipycanvas
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import io
import concurrent.futures
import asyncio
import traceback
import random
from _utilities import image_to_text
from _endpoints import prompt_claude, prompt_chatgpt, prompt_gemini

In [2]:
# vision language models to use
use_gpt4o = True
use_claude = True
use_gemini = True

# List of words to draw
words_list = ['car', 'cat', 'house', 'dog', 'tree', 'sun', 'moon', 'star', 'bird', 'fish',
              'flower', 'book', 'chair', 'table', 'phone', 'clock', 'shoe', 'hat', 'cup', 'ball',
              'apple', 'banana', 'bear', 'bike', 'boat', 'bottle', 'box', 'bridge', 'brush', 'bus',
              'butterfly', 'camera', 'candle', 'cloud', 'coin', 'cow', 'crown', 'desk', 'door', 'duck',
              'egg', 'elephant', 'eye', 'face', 'fan', 'flag', 'frog', 'gift', 'glass', 'guitar',
              'hand', 'heart', 'horse', 'key', 'king', 'lamp', 'leaf', 'leg', 'lion', 'lock',
              'map', 'monkey', 'moon', 'mouse', 'mouth', 'nose', 'pen', 'pig', 'plane', 'plant',
              'rabbit', 'rain', 'ring', 'robot', 'rocket', 'rope', 'rose', 'santa', 'ship', 'snake',
              'snow', 'sock', 'spider', 'spoon', 'square', 'star', 'sun', 'sword', 'table', 'tent',
              'tiger', 'train', 'tree', 'truck', 'umbrella', 'vase', 'watch', 'wheel', 'window', 'zebra']

# available colors for drawing
colors = ['red', 'blue', 'green', 'yellow', 'orange', 
          'purple', 'brown', 'pink', 'gray', 'black']

### Create canvas
canvas = ipycanvas.Canvas(width=256, height=256)
canvas.stroke_style = "black"
canvas.line_width = 2
canvas.sync_image_data = True

# Add black frame
def draw_frame():
    canvas.stroke_rect(0, 0, 256, 256)
draw_frame()

### Create color buttons and text panel
buttons = [widgets.Button(description='', 
                         layout=widgets.Layout(width='30px', height='30px'),
                         style=dict(button_color=c)) for c in colors]
text_panel = widgets.HTML(
    value="",
    layout=widgets.Layout(width='256px', height='256px')
)

### Variables for drawing
drawing = False
last_x = 0
last_y = 0
pil_image = None
responses = [""]

### Events
def on_mouse_down(x, y):
    global drawing, last_x, last_y
    drawing = True
    last_x = x
    last_y = y

def on_mouse_move(x, y):
    global drawing, last_x, last_y
    if drawing:
        canvas.begin_path()
        canvas.move_to(last_x, last_y)
        canvas.line_to(x, y)
        canvas.stroke()
        last_x = x
        last_y = y


def on_mouse_up(x, y):
    global drawing, canvas, pil_image, responses

    drawing = False
    if responses[-1].endswith("wins!"):
        return

    # workaround to complete drawing in image
    canvas.stroke_circle(0,0,1)
    canvas.stroke_circle(0,0,1)
    canvas.flush()
    
    pil_image = image_to_text(Image.open(io.BytesIO(canvas.image_data)))

    try:
        # Execute and print results as they come 
        for result in parallel_execution(pil_image):
            add_response(result)
    except Exception as e:
        error_traceback = traceback.format_exc()
        add_response("error" + error_traceback)

def on_color_click(b):
    canvas.stroke_style = b.style.button_color
    draw_frame()  # redraw frame after color change

### Connect events
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

for button in buttons:
    button.on_click(on_color_click)

# Add new function before Layout section
def clear_canvas():
    canvas.fill_style = 'white'
    canvas.fill_rect(0, 0, 256, 256)
    canvas.fill_style = canvas.stroke_style  # restore current color
    draw_frame()

def on_next_click(b):
    global target_text, responses
    target_text.value = random.choice(words_list)
    responses = [""]
    add_response("")
    clear_canvas()

def former_responses():
    global responses
    return ",".join([r.strip().strip(".").split(" ")[-1] for r in responses])

def ask_gpt4o(pil_image):
    its_not = former_responses()
    return "GPT-4o: " + prompt_chatgpt(f"What's drawn in this image? It's not: {its_not}. Answer one word only.", image=pil_image)

def ask_claude(pil_image):
    its_not = former_responses()
    return "Claude: " + prompt_claude(f"What's drawn in this image? It's not: {its_not}.  Answer one word only.", image=pil_image)

def ask_gemini(pil_image):
    its_not = former_responses()
    return "Gemini: " + prompt_gemini(f"What's drawn in this image? It's not: {its_not}.  Answer one word only.", image=pil_image)

ask_functions = []
if use_gpt4o: 
    ask_functions.append(ask_gpt4o)
if use_claude:
    ask_functions.append(ask_claude)
if use_gemini:
    ask_functions.append(ask_gemini)

def parallel_execution(pil_image):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_func = {executor.submit(func, pil_image): func for func in ask_functions}
        for future in concurrent.futures.as_completed(future_to_func):
            yield future.result()

def add_response(text):
    """Add a new entry to the output text field"""
    global text_panel, responses, target_text
    if responses[-1].endswith("wins!"):
        return
        
    # Check if this is a LLM response (contains ":") and if the last word matches
    if ":" in text and text.split()[-1].lower().strip().strip(".") == target_text.value.lower():
        responses.append('<div style="font-color:red;"><b>' + text.replace(":", ":</b>") + "</div>")
        responses.append(text.split(":")[0] + " wins!")
    else:
        responses.append("<b>" + text.replace(":", ":</b>"))
    
    text_panel.value = '<div style="font-size:20px;"><br/>' + "<br/>".join(responses[::-1][:10]) + "</div>"

### Graphical User Interface
target_label = widgets.Label(value="Only you see this. Please draw a ")
target_text = widgets.Text(value=random.choice(words_list), layout=widgets.Layout(width='100px'))
next_button = widgets.Button(description="Next")
next_button.on_click(on_next_click)

target_box = widgets.HBox([target_label, target_text, next_button])
colors_box = widgets.HBox(buttons)
panels_box = widgets.VBox([text_panel])
display(widgets.HBox([
    widgets.VBox([colors_box, canvas, target_box]),
    panels_box
]))